In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

### Héja-galamb játék kétdimenziós négyzetrácson

#### Játék alapjai:
- négyzetrács elkészítése
- adott rácspont szomszédainak keresése
- adott rácspontban levő egyed lehetséges stratégiáinak kifizetési értékeinek számolása, majd innen megadható, hogy melyik stratégia öröklődik
- játék indítása megválasztott kezdeti feltételből
- játék iterálása, érdekes paraméterek mentése

#### Paraméterek:
- kezdetben H és D stratégiák azonos arányban vannak, véletlenszerű eloszlással
- rács mérete 70x70
- a szimuláció időtartama 175 generáció minden $\beta$ paraméter mellett
- utolsó 25 generáció ábrázolása

In [ ]:
def moore_neighbourhood( x, y, size_x, size_y ):
    """
    Moore szomszédság indexeit adja vissza periodikus határfeltétellel.
    https://mathworld.wolfram.com/MooreNeighborhood.html
    
    Paraméterek:
    -x: a szimulációs tábla indexe (koordinátája) az x tengelyen,
    -y: a szimulációs tábla indexe (koordinátája) az y tengelyen,
    -size_x: a szimulációs tábla mérete x irányban,
    -size_y: a szimulációs tábla mérete y irányban.
    """
    current = np.array( [ [x, y] ] )
    neigh_idxs = np.array( [ [ x-1, y+1 ], [ x, y+1 ], [ x+1, y+1 ],
                           [ x-1, y ], [ x+1, y ],
                           [ x-1, y-1], [ x, y-1 ], [ x+1, y-1 ] ] )
    neigh_idxs[:,0] %= size_x
    neigh_idxs[:,1] %= size_y    

    return current, neigh_idxs

In [ ]:
def von_neumann_neighbourhood( x, y, size_x, size_y ):
    """
    Von Neumann szomszédság indexeit adja vissza periodikus határfeltétellel.
    https://mathworld.wolfram.com/vonNeumannNeighborhood.html
    
    Paraméterek:
    -x: a szimulációs tábla indexe (koordinátája) az x tengelyen,
    -y: a szimulációs tábla indexe (koordinátája) az y tengelyen,
    -size_x: a szimulációs tábla mérete x irányban,
    -size_y: a szimulációs tábla mérete y irányban.
    """
    current = np.array( [ [x, y] ] )
    neigh_idxs = np.array( [ [ x, y+1 ],
                           [ x-1, y ], [ x+1, y ],
                           [ x, y-1 ] ] )
    neigh_idxs[:,0] %= size_x
    neigh_idxs[:,1] %= size_y
    return current, neigh_idxs

In [ ]:
def generate_table( size_x, size_y, num_strats=2 ):
    """
    Stratégiák kódolása:
        0: héja,
        1: galamb,
        
    Paraméterek:
    -size_x: a szimulációs tábla mérete x irányban,
    -size_y: a szimulációs tábla mérete y irányban,
    -num_strats: különböző stratégiák száma a játékban.
    """
    table = np.zeros( (size_x*size_y), dtype=int )
    np.random.seed(0)
    rnd_idxs = np.random.permutation( size_x*size_y )
    for i in range( num_strats ):
        table[ rnd_idxs[i::num_strats] ] = i
    return table.reshape( size_x, size_y )

In [ ]:
def iterate( table, payoff_mat, neighbourhood_func ):
    """
    Következő generációba lépteti a táblát.
    
    Paraméterek:
    -table: a szimulációs tábla (négyzetrács),
    -payoff_mat: a szimulációban szereplő kifizetési mátrix,
    -neighbourhood_func: szomszédságot definiáló függvény
                        (jelenleg Moore vagy Von-Neumann)    
    """
    table_s = np.zeros( table.shape, dtype=np.float64 )
    table_new = np.zeros( table.shape, dtype=int )
    
    for i in range( table.shape[0] ):
        for j in range( table.shape[1] ):
            current, neighs = neighbourhood_func(i, j, table.shape[0], table.shape[1] ) # indexek
            
            current_strat = table[ current[0,0], current[0,1] ] 
            neighs_strat = table[ neighs[:,0], neighs[:,1] ] # héja 0, galamb 1, egyéb..
            
            current_s_value = 0
            for k in range( neighs_strat.shape[0] ): # szomszédok
                table_s[i,j] += payoff_mat[ current_strat, neighs_strat[k] ]
    
    for i in range( table.shape[0] ):
        for j in range( table.shape[1] ):
            
            # mostani és szomszédainak koordinátái:
            current, neighs = neighbourhood_func(i, j, table.shape[0], table.shape[1] )
            
            # összes szomszéd és saját koordináták:
            all_neighs = np.concatenate( (current, neighs), axis=0 )
            # összes szomszédok és saját fitnesz értékek:
            neighs_s = table_s[ all_neighs[:,0], all_neighs[:,1] ]
            
            max_val = np.max( neighs_s ) # maximum fitnesz érték
            max_idx = np.argwhere( np.isclose(neighs_s, max_val) ).flatten() # hányszor fordul elő
            rnd_idx = np.random.permutation( max_idx.shape[0] )[0] # random választás
            max_idx_chosen = max_idx[ rnd_idx ]
            
            local_max_s_coords = all_neighs[ max_idx_chosen ] # max s szomszéd koordinátája
            table_new[i,j] = table[ local_max_s_coords[0], local_max_s_coords[1] ]
    
    return table_new

#### Szomszédságkereső függvények tesztelése (periodikus határfeltétellel)

##### Von-Neumann szomszédság a (0,0) elem esetében

In [ ]:
table = generate_table( 10, 10 )
plt.figure()
plt.pcolormesh( table )
current, neighs = von_neumann_neighbourhood( 0, 0, table.shape[0], table.shape[1] )
table[ neighs[:,0], neighs[:,1] ] = 2
plt.figure()
plt.pcolormesh( table )

##### Moore szomszédság a (9,9) elem esetében.

In [ ]:
table = generate_table( 10, 10 )
plt.figure()
plt.pcolormesh( table )
current, neighs = moore_neighbourhood( 9, 9, table.shape[0], table.shape[1] )
table[ neighs[:,0], neighs[:,1] ] = 2
plt.figure()
plt.pcolormesh( table )

#### Játék!

##### beta értékek listája, melyeknél fut a szimuláció

In [ ]:
beta_list = np.array( [ 0.9, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4,
                        1.45, 1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8,
                        1.85, 1.9, 1.95, 2.0, 2.1, 2.2, 2.3, 2.5, 2.7, 3.0 ] )
beta_list

##### játéktáblák mentése

In [ ]:
table_m = []
for m in tqdm( range( beta_list.shape[0] ) ):
    table_all = []

    table = generate_table( 70, 70 ) # tábla
    beta = beta_list[m] # C/V paraméter
    payoff = np.array( [ [1-beta, 2], [0, 1] ] ) # kifizetési mátrix
    table_all.append( np.copy(table) ) # kezdeti állapot mentése

    for n in range( 175 ):
        table = iterate( table, payoff, moore_neighbourhood )
        table_all.append( np.copy(table) )
    table_m.append( np.array( table_all ) )
table_m = np.concatenate( np.expand_dims(table_m, axis=0) )
table_m.shape

##### ábrázolás

In [ ]:
hawk_fractions = np.zeros( (beta_list.shape[0], 25), dtype=np.float64 )
for k in range( beta_list.shape[0] ):
    for l in range( 1, 26 ):
        uniq, counts = np.unique( table_m[k][-l], return_counts=True )
        hawk_fractions[k, l-1] = counts[0] / counts.sum()

In [ ]:
plt.figure( figsize=(10,6), dpi=100 )
plt.scatter( np.ones(25)*beta_list[0], hawk_fractions[0], c='k', s=10, label='térbeli' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(25)*beta_list[k], hawk_fractions[k], c='k', s=15, )
    
classic_result = np.ones( beta_list.shape )
classic_result[ beta_list > 1 ] = 1/beta_list[ beta_list > 1 ]
plt.scatter( beta_list, classic_result, c='maroon', s=10, marker='x', label='klasszikus' )

plt.vlines( x=9/7, ymin=0.2, ymax=1.0, colors='forestgreen', linestyles='dashed', 
            label='1. fázisátalakulás' )
plt.vlines( x=5/3, ymin=0.2, ymax=1.0, colors='navy', linestyles='dashed',
           label='2. fázisátalakulás' )
plt.vlines( x=2.0, ymin=0.2, ymax=1.0, colors='blue', linestyles='dashed',
           label='3. fázisátalakulás' )

plt.xlabel(r'$ \beta $', fontsize=22)
plt.ylabel("Héja stratégiák aránya", fontsize=22)
plt.tick_params( labelsize=18 )
plt.legend()
plt.savefig('heja_galamb_Moore.png', dpi=100)

#### Von-Neumann szomszédsággal

In [ ]:
table_m = []
for m in tqdm( range( beta_list.shape[0] ) ):
    table_all = []

    table = generate_table( 70, 70 ) # tábla
    beta = beta_list[m] # C/V paraméter
    payoff = np.array( [ [1-beta, 2], [0, 1] ] ) # kifizetési mátrix
    table_all.append( np.copy(table) ) # kezdeti állapot mentése

    for n in range( 175 ):
        table = iterate( table, payoff, von_neumann_neighbourhood )
        table_all.append( np.copy(table) )
    table_m.append( np.array( table_all ) )
table_m = np.concatenate( np.expand_dims(table_m, axis=0) )
table_m.shape

In [ ]:
hawk_fractions = np.zeros( (beta_list.shape[0], 25), dtype=np.float64 )
for k in range( beta_list.shape[0] ):
    for l in range( 1, 26 ):
        uniq, counts = np.unique( table_m[k][-l], return_counts=True )
        hawk_fractions[k, l-1] = counts[0] / counts.sum()

In [ ]:
plt.figure( figsize=(10,6), dpi=100 )
plt.scatter( np.ones(25)*beta_list[0], hawk_fractions[0], c='k', s=10, label='térbeli' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(25)*beta_list[k], hawk_fractions[k], c='k', s=15, )
    
classic_result = np.ones( beta_list.shape )
classic_result[ beta_list > 1 ] = 1/beta_list[ beta_list > 1 ]
plt.scatter( beta_list, classic_result, c='maroon', s=10, marker='x', label='klasszikus' )

plt.vlines( x=1.025, ymin=0.2, ymax=1.0, colors='forestgreen', linestyles='dashed', 
            label='1. fázisátalakulás' )
plt.vlines( x=1.325, ymin=0.2, ymax=1.0, colors='navy', linestyles='dashed',
           label='2. fázisátalakulás' )
plt.vlines( x=1.675, ymin=0.2, ymax=1.0, colors='blue', linestyles='dashed',
           label='3. fázisátalakulás' )

plt.xlabel(r'$ \beta $', fontsize=22)
plt.ylabel("Héja stratégiák aránya", fontsize=22)
plt.tick_params( labelsize=18 )
plt.legend()
plt.savefig('heja_galamb_vonN.png', dpi=100)

#### Keverés minden generációban

In [ ]:
table_m = []
for m in tqdm( range( beta_list.shape[0] ) ):
    table_all = []
    
    table = generate_table( 70, 70 ) # tábla
    beta = beta_list[m] # C/V paraméter
    payoff = np.array( [ [1-beta, 2], [0, 1] ] ) # kifizetési mátrix
    table_all.append( np.copy(table) ) # kezdeti állapot mentése

    for n in range( 175 ):
        table = iterate( table, payoff, moore_neighbourhood )
        np.random.seed(n)
        shuffle_idx = np.random.permutation( table.flatten().shape[0] )
        table = np.copy(table.flatten())[shuffle_idx].reshape( table.shape[0], table.shape[1] )
        table_all.append( np.copy(table) )
    table_m.append( np.array( table_all ) )
table_m = np.concatenate( np.expand_dims(table_m, axis=0) )
table_m.shape

In [ ]:
hawk_fractions = np.zeros( (beta_list.shape[0], 25), dtype=np.float64 )
for k in range( beta_list.shape[0] ):
    for l in range( 1, 26 ):
        uniq, counts = np.unique( table_m[k][-l], return_counts=True )
        hawk_fractions[k, l-1] = counts[0] / counts.sum()

In [ ]:
plt.figure( figsize=(10,6), dpi=100 )
plt.scatter( np.ones(25)*beta_list[0], hawk_fractions[0], c='k', s=10, label='térbeli' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(25)*beta_list[k], hawk_fractions[k], c='k', s=10, )
    
classic_result = np.ones( beta_list.shape )
classic_result[ beta_list > 1 ] = 1/beta_list[ beta_list > 1 ]
plt.scatter( beta_list, classic_result, c='maroon', s=10, marker='x', label='klasszikus' )

plt.xlabel(r'$ \beta $', fontsize=22)
plt.ylabel("Héja stratégiák aránya", fontsize=22)
plt.tick_params( labelsize=18 )
plt.legend()
plt.savefig('heja_galamb_kevert_Moore.png', dpi=100)

#### Von-Neumann szomszédsággal

In [ ]:
table_m = []
for m in tqdm( range( beta_list.shape[0] ) ):
    table_all = []
    
    table = generate_table( 70, 70 ) # tábla
    beta = beta_list[m] # C/V paraméter
    payoff = np.array( [ [1-beta, 2], [0, 1] ] ) # kifizetési mátrix
    table_all.append( np.copy(table) ) # kezdeti állapot mentése

    for n in range( 175 ):
        table = iterate( table, payoff, von_neumann_neighbourhood )
        np.random.seed(n)
        shuffle_idx = np.random.permutation( table.flatten().shape[0] )
        table = np.copy(table.flatten())[shuffle_idx].reshape( table.shape[0], table.shape[1] )
        table_all.append( np.copy(table) )
    table_m.append( np.array( table_all ) )
table_m = np.concatenate( np.expand_dims(table_m, axis=0) )
table_m.shape

In [ ]:
hawk_fractions = np.zeros( (beta_list.shape[0], 25), dtype=np.float64 )
for k in range( beta_list.shape[0] ):
    for l in range( 1, 26 ):
        uniq, counts = np.unique( table_m[k][-l], return_counts=True )
        hawk_fractions[k, l-1] = counts[0] / counts.sum()

In [ ]:
plt.figure( figsize=(10,6), dpi=100 )
plt.scatter( np.ones(25)*beta_list[0], hawk_fractions[0], c='k', s=10, label='térbeli' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(25)*beta_list[k], hawk_fractions[k], c='k', s=10, )
    
classic_result = np.ones( beta_list.shape )
classic_result[ beta_list > 1 ] = 1/beta_list[ beta_list > 1 ]
plt.scatter( beta_list, classic_result, c='maroon', s=10, marker='x', label='klasszikus' )

plt.xlabel(r'$ \beta $', fontsize=22)
plt.ylabel("Héja stratégiák aránya", fontsize=22)
plt.tick_params( labelsize=18 )
plt.legend()
plt.savefig('heja_galamb_kevert_vonN.png', dpi=100)

### Héja-galamb-megtorló játék kétdimenziós négyzetrácson

##### játéktáblák mentése

In [ ]:
table_m = []
for m in tqdm( range( beta_list.shape[0] ) ):
    table_all = []
    
    table = generate_table( 70, 70, num_strats=3 ) # tábla
    beta = beta_list[m] # C/V paraméter
    eps = 0.0
    payoff = np.array( [ [ 1-beta, 2, 1-beta+eps ], 
                         [ 0, 1, 1-eps ], 
                         [ 1-beta-eps, 1+eps, 1 ] ] ) # kifizetési mátrix
    table_all.append( np.copy(table) ) # kezdeti állapot mentése

    for n in range( 175 ):
        table = iterate( table, payoff, moore_neighbourhood )
        table_all.append( np.copy(table) )
    table_m.append( np.array( table_all ) )
table_m = np.concatenate( np.expand_dims(table_m, axis=0) )
table_m.shape

In [ ]:
lastnum = 25 # mennyi utolsó generáció legyen az ábrán
hawk_fractions = np.zeros( (beta_list.shape[0], lastnum), dtype=np.float64 )
dove_fractions = np.zeros( (beta_list.shape[0], lastnum), dtype=np.float64 )
revenge_fractions = np.zeros( (beta_list.shape[0], lastnum), dtype=np.float64 )
for k in range( beta_list.shape[0] ):
    for l in range( 1, lastnum+1 ):
        hawk_fractions[k, l-1] = (table_m[k,l] == 0).sum() / (table_m.shape[-2]*table_m.shape[-1])
        dove_fractions[k, l-1] = (table_m[k,l] == 1).sum() / (table_m.shape[-2]*table_m.shape[-1])
        revenge_fractions[k, l-1] = (table_m[k,l] == 2).sum() / (table_m.shape[-2]*table_m.shape[-1])

In [ ]:
plt.figure( figsize=(10,6), dpi=100 )
plt.scatter( np.ones(lastnum)*beta_list[0], hawk_fractions[0], c='maroon', s=10, label='Héja' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(lastnum)*beta_list[k], hawk_fractions[k], c='maroon', s=10, )
    
    
plt.scatter( np.ones(lastnum)*beta_list[0], dove_fractions[0], c='navy', s=10, label='Galamb' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(lastnum)*beta_list[k], dove_fractions[k], c='navy', s=10, )

plt.scatter( np.ones(lastnum)*beta_list[0], revenge_fractions[0], c='forestgreen', s=10, label='Megtorló' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(lastnum)*beta_list[k], revenge_fractions[k], c='forestgreen', s=10, )
    

plt.xlabel(r'$ \beta $', fontsize=22)
plt.ylabel("Héja stratégiák aránya", fontsize=22)
plt.tick_params( labelsize=18 )
plt.legend()
plt.savefig('heja_galamb_megtorlo_Moore.png', dpi=100)

In [ ]:
table_m = []
for m in tqdm( range( beta_list.shape[0] ) ):
    table_all = []
    
    table = generate_table( 70, 70, num_strats=3 ) # tábla
    beta = beta_list[m] # C/V paraméter
    eps = 0.0
    payoff = np.array( [ [ 1-beta, 2, 1-beta+eps ], 
                         [ 0, 1, 1-eps ], 
                         [ 1-beta-eps, 1+eps, 1 ] ] ) # kifizetési mátrix
    table_all.append( np.copy(table) ) # kezdeti állapot mentése

    for n in range( 175 ):
        table = iterate( table, payoff, von_neumann_neighbourhood )
        table_all.append( np.copy(table) )
    table_m.append( np.array( table_all ) )
table_m = np.concatenate( np.expand_dims(table_m, axis=0) )
table_m.shape

In [ ]:
lastnum = 25 # mennyi utolsó generáció legyen az ábrán
hawk_fractions = np.zeros( (beta_list.shape[0], lastnum), dtype=np.float64 )
dove_fractions = np.zeros( (beta_list.shape[0], lastnum), dtype=np.float64 )
revenge_fractions = np.zeros( (beta_list.shape[0], lastnum), dtype=np.float64 )
for k in range( beta_list.shape[0] ):
    for l in range( 1, lastnum+1 ):
        hawk_fractions[k, l-1] = (table_m[k,l] == 0).sum() / (table_m.shape[-2]*table_m.shape[-1])
        dove_fractions[k, l-1] = (table_m[k,l] == 1).sum() / (table_m.shape[-2]*table_m.shape[-1])
        revenge_fractions[k, l-1] = (table_m[k,l] == 2).sum() / (table_m.shape[-2]*table_m.shape[-1])

In [ ]:
plt.figure( figsize=(10,6), dpi=100 )
plt.scatter( np.ones(lastnum)*beta_list[0], hawk_fractions[0], c='maroon', s=10, label='Héja' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(lastnum)*beta_list[k], hawk_fractions[k], c='maroon', s=10, )
    
    
plt.scatter( np.ones(lastnum)*beta_list[0], dove_fractions[0], c='navy', s=10, label='Galamb' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(lastnum)*beta_list[k], dove_fractions[k], c='navy', s=10, )

plt.scatter( np.ones(lastnum)*beta_list[0], revenge_fractions[0], c='forestgreen', s=10, label='Megtorló' )    
for k in range( 1, beta_list.shape[0] ):
    plt.scatter( np.ones(lastnum)*beta_list[k], revenge_fractions[k], c='forestgreen', s=10, )
    

plt.xlabel(r'$ \beta $', fontsize=22)
plt.ylabel("Héja stratégiák aránya", fontsize=22)
plt.tick_params( labelsize=18 )
plt.legend()
plt.savefig('heja_galamb_megtorlo_vonN.png', dpi=100)